In [ ]:
import numpy as np
import pandas as pd
import time

In [ ]:
data = pd.read_csv("weather_classification_data.csv")

for name in data.columns[0:len(data.columns)]:
    unique_vals = data[f'{name}'].unique()
    if len(unique_vals) < 10: # fazla tür olmadığını biliyorum
        mapping = {val: idx for idx, val in enumerate(unique_vals)}
        data[f'{name}'] = data[f'{name}'].map(mapping)



In [ ]:
from sklearn.model_selection import train_test_split
X = data.drop('Weather Type', axis=1).astype(np.float64)
y = data.iloc[:, -1].astype(np.int64)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1233)


<h1>PYTHON İLE KENDIMİZ YAZALIM</h1>

In [ ]:
class Gaussian_NaiveBayes:

    def fit(self,X,y):
            
        n_samples, n_features = X.shape #satır sütun sayılarını alalım
        self._classes = np.unique(y)
        n_classes = len(self._classes)

        #Her sınıf için ortalama, varyans ve öncelik hesaplayalım
        self._mean = np.zeros((n_classes,n_features), dtype=np.float64)
        self._varience = np.zeros((n_classes,n_features), dtype=np.float64)
        self._priors = np.zeros(n_classes, dtype=np.float64)

        for idx, c in enumerate(self._classes):
            X_class = X[y==c] # Her sınıfın verisiyle ayrı ayrı işlem yapmak için X_class değişkenini kullanıyoruz, "y==c" y dataframe'indeki c sınıfından olan satırları almamızı sağlar
            self._mean[idx,:] = X_class.mean(axis=0) # Seçilen sütunun ortalaması alınır
            self._varience[idx,:] = X_class.var(axis=0)
            self._priors[idx] = X_class.shape[0]/float(n_samples) # Her sınıfın frekansı bulunur ve toplam örnek sayısına bölünür
            
    def predict(self, X):
        y_pred = [self._predict(np.array(row[1:])) for row in X.itertuples()]
        return np.array(y_pred)

    def _predict(self, x):
        posteriors = [] # Sonsal olasılık değerlerini tutmak için

        # Her sınıf için sonsal olasılığı(posterior probablity) hesaplayalım
        for idx, c in enumerate(self._classes):
            prior = np.log(self._priors[idx])
            posterior = np.sum(np.log(self._pdf(idx,x))) 
            posterior = posterior + prior  
            posteriors.append(posterior)

        # Sonsal olasılığı en büyük olan sınıfı döndür
        return self._classes[np.argmax(posteriors)]
        
    def _pdf(self,class_idx,x): # Gaussian Probablity Density Function(Olasılık Yoğunluk Fonksiyonu) hesaplama
         mean = self._mean[class_idx]
         var = self._varience[class_idx]
         numerator = np.exp(-(x-mean)**2 / (2 * var)) # formülün payı
         denominator = np.sqrt(2*np.pi * var) # formülün paydası
        
         return numerator / denominator


In [ ]:
start_time = time.time()

# Model eğitimi ve tahmin
nb = Gaussian_NaiveBayes()
nb.fit(X_train,y_train)
predictions = nb.predict(X_test)

end_time = time.time()
net_zaman = end_time - start_time
print(f"Geçen zaman: {net_zaman:.4f} saniye")

In [ ]:
# Confusion matrix hesaplama:
cm = confusion_matrix(y_test, predictions)

# Confusion matrix'i görselleştirme:
plt.figure(figsize=(6, 5))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.colorbar()

# Sınıf etiketlerini belirleme
classes = np.unique(y)
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)

# Her hücreyi annotasyon ekleyerek gösterme:
thresh = cm.max() / 2.0
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

plt.ylabel('Gerçek Etiket')
plt.xlabel('Tahmin Edilen Etiket')
plt.tight_layout()
plt.show()

# Metrik hesaplamaları:
accuracy = accuracy_score(y_test, predictions)
# Çok sınıflı problemde recall ve f1 için average parametresi verilmeli.
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')

print("Accuracy:", accuracy)
print("Recall:", recall)
print("F1 Score:", f1)